In [1]:
import numpy as np
from sklearn.datasets import fetch_openml
from scipy.fftpack import fft
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, f1_score
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from tqdm import tqdm
import math
from sklearn.metrics.pairwise import cosine_similarity
import scienceplots
import matplotlib as mpl
from copy import deepcopy
mpl.rcParams['figure.dpi'] = 300

In [2]:
import lib.python.lib_FHRR_Loihi as lib
import lib.python.utility as util
from lava.magma.core.run_configs import Loihi1SimCfg
from lava.magma.core.run_configs import Loihi2HwCfg
from lava.magma.core.run_conditions import RunSteps

In [3]:
def plot_multiple_box_plots(data_sets, x_title="X Values", y_title="Y Values", title="", show_box_plots=True, label_prefix="", y_limits=None, x_limits=None, figsize=(10, 6), show_outliers=False):
    """
    Plots multiple sets of customized black and white box plots for each x value with multiple y values,
    indicates outliers, and adds a line graph connecting the means of y values for each dataset.

    Parameters:
    data_sets (list of dicts): List of dictionaries, each containing 'x_values', 'y_values', and 'title' keys.
                               Each dictionary represents a dataset.
    x_title (str): Label for the x-axis.
    y_title (str): Label for the y-axis.
    title (str): Overall title of the plot.
    show_box_plots (bool): If True, display box plots. If False, only plot the means.
    y_limits (tuple): Optional parameter to set the y-axis limits. Should be a tuple (y_min, y_max).
    """
    fig, ax = plt.subplots(figsize=figsize)

    plt.rcParams["figure.figsize"] = [7.00, 7.50]
    plt.rcParams["figure.autolayout"] = True

    colors = ['black', 'blue', 'green', 'red', 'purple', 'orange']  # Add more colors as needed

    for idx, data_set in enumerate(data_sets):
        x_values = data_set['x_values']
        y_values = data_set['y_values']
        dataset_title = data_set.get('title', f'Dataset {idx + 1}')
        color = colors[idx % len(colors)]
        
        if show_box_plots:
            # Create the box plot
            boxplot = ax.boxplot(y_values, positions=x_values, patch_artist=False, showmeans=True, meanline=True,
                                 showfliers=show_outliers,
                                 flierprops=dict(marker='x', color=color, markersize=5), boxprops=dict(color=color),
                                 whiskerprops=dict(color=color), capprops=dict(color=color), medianprops=dict(color=color))
        
        # Calculate means of y values
        means = [np.mean(y) for y in y_values]
        
        # Plot the line graph connecting the means
        label_txt = label_prefix + f' {dataset_title}'
        ax.plot(x_values, means, color=color, linestyle='-', marker='o', markerfacecolor='none', label=label_txt)

    # Set the x-axis labels
    ax.set_xticks(x_values)
    ax.set_xticklabels([str(x) for x in x_values])

    # Set labels and title
    ax.set_xlabel(x_title)
    ax.set_ylabel(y_title)
    ax.set_title(title)

    # Set y-axis limits if provided
    if y_limits is not None:
        ax.set_ylim(y_limits)

    if x_limits is not None:
        ax.set_xlim(x_limits)

    # Show the legend
    ax.legend()

    # Show the plot
    plt.show()


In [5]:
# Encoding
def rate_encode(values, duration):
    # Normalize values to [0, 1] range
    min_val = np.min(values)
    max_val = np.max(values)
    normalized_values = (values - min_val) / (max_val - min_val)

    # Generate Poisson spike trains
    spike_trains = np.random.binomial(1, normalized_values, (duration, len(values)))
    return spike_trains, min_val, max_val

# Decoding
def rate_decode(spike_trains, duration, min_val, max_val):
    # Count spikes and average over duration
    spike_counts = np.sum(spike_trains, axis=0)
    decoded_values = spike_counts / duration

    # Denormalize values back to original range
    original_values = decoded_values * (max_val - min_val) + min_val
    return original_values

def rate_decode_get_activity(spike_trains, duration, min_val, max_val):
    spike_counts = np.sum(np.sum(spike_trains, axis=0))
    return spike_counts

# Encoding
def latency_encode(values, duration):
    # Normalize values to [0, 1] range
    min_val = np.min(values)
    max_val = np.max(values)
    normalized_values = (values - min_val) / (max_val - min_val)
    
    # Generate spike trains based on latency
    spike_trains = np.zeros((duration, len(values)))
    for i, value in enumerate(normalized_values):
        spike_time = int((1 - value) * (duration - 1))
        spike_trains[spike_time, i] = 1
    return spike_trains, min_val, max_val

# Decoding
def latency_decode(spike_trains, duration, min_val, max_val):
    # Find the first spike time for each neuron
    first_spike_times = np.argmax(spike_trains, axis=0)
    
    # Normalize back to [0, 1] range
    normalized_values = 1 - (first_spike_times / (duration - 1))
    
    # Denormalize values back to original range
    original_values = normalized_values * (max_val - min_val) + min_val
    return original_values

def preprocess_latency_encode_vec_TTFS(vec, v_th, sim_time):
    # [f(x) if x is not None else '' for x in xs]
    # vec = np.array([v_th * 4  if (2 * math.pi * v_th) / (sim_time * (elem)) > v_th else elem for elem in vec  ])
    return v_th * (2* math.pi) / (vec * sim_time)

def preprocess_latency_decode_vec_TTFS(vec, v_th, sim_time):
    # [f(x) if x is not None else '' for x in xs]
    # vec = np.array([v_th * 4  if (2 * math.pi * v_th) / (sim_time * (elem)) > v_th else elem for elem in vec  ])
    return (v_th * 2*math.pi) / (sim_time * vec)
    

def preprocess_latency_encode_vec_Phase(vec, v_th, sim_time):
    # [f(x) if x is not None else '' for x in xs]
    # vec = np.array([v_th * 4  if (2 * math.pi * v_th) / (sim_time * (elem)) > v_th else elem for elem in vec  ])
    return v_th * (vec) / ((2* math.pi) * sim_time)

In [10]:
# Example usage
values = np.array([10, 0.5, 0.81, 0.3, 12])  # Example d-dimensional array
duration = 100

# Encode
spike_trains, min_val, max_val = rate_encode(values, duration)
# print("Spike Trains:\n", spike_trains)

# Decode
decoded_values = rate_decode(spike_trains, duration, min_val, max_val)
print("Decoded Values:\n", decoded_values)

Decoded Values:
 [ 9.894  0.417  0.885  0.3   12.   ]


In [11]:
rate_decode_get_activity(spike_trains, duration, min_val, max_val)

188